This notebook is a *heavily* adapted version of https://github.com/habi/EAWAG/blob/main/DataWrangling.ipynb, specifically for the talk at the Institute seminar in 2024.

In [ ]:
# Load the python modules we need
import platform
import os
import glob
import pandas
import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
from dask.distributed import Client, LocalCluster
import skimage
from tqdm import notebook

In [ ]:
# Load our own log file parsing code
from BrukerSkyScanLogfileRuminator.parsing_functions import *

In [ ]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    # Check if me mounted the FastSSD, otherwise go to standard tmp file
    if os.path.exists(os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')):
        tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD', 'tmp')
    else:
        tmp = tempfile.gettempdir()
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\tmp')
    else:
        tmp = os.path.join('D:\\tmp')
dask.config.set({'temporary_directory': tmp})
print('Dask temporary files go to %s' % dask.config.get('temporary_directory'))

In [ ]:
from dask.distributed import Client
client = Client()

In [ ]:
client

In [ ]:
print('You can see what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

In [ ]:
# Set up figure defaults for the talk
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['savefig.transparent'] = True
# Set seaborn theme
seaborn.set_theme(
    context='talk',
    style='whitegrid',
)
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation

In [ ]:
Root = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD', 'EAWAG')
print('We are loading all the data from %s' % Root)

In [ ]:
# Make directory for output
# For these notebooks, we simply dump the images one directory up into the relevant 'media' folder
OutPutDir = os.path.join('..', 'media', 'EAWAG')
print('We are saving all the output to %s' % OutPutDir)
os.makedirs(OutPutDir, exist_ok=True)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files, unsorted but fast
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [ ]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [ ]:
# Get rid of all non-rec logfiles
for c, row in Data.iterrows():
    if 'rec' not in row.Folder:
        Data.drop([c], inplace=True)
    elif 'rectmp.log' in row.LogFile:
        Data.drop([c], inplace=True)
# Reset dataframe index
Data = Data.reset_index(drop=True)

In [ ]:
# Generate us some meaningful colums
Data['Fish'] = [l[len(Root) + 1:].split(os.sep)[0] for l in Data['LogFile']]
Data['Scan'] = ['.'.join(l[len(Root) + 1:].split(os.sep)[1:-1]) for l in Data['LogFile']]

In [ ]:
# Get parameters related to scan from logfiles
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]
Data['Exposuretime'] = [exposuretime(log) for log in Data['LogFile']]
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]
Data['Scan date'] = [scandate(log) for log in Data['LogFile']]
Data['ProjectionSize'] = [projection_size(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['ThreeSixty'] = [threesixtyscan(log) for log in Data['LogFile']]

In [ ]:
Data['Voxelsize'].unique()

In [ ]:
# Get parameters related to reconstruction from logfiles
Data['ReconstructionSize'] = [reconstruction_size(log) for log in Data['LogFile']]
Data['Grayvalue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [ringremoval(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [beamhardening(log) for log in Data['LogFile']]
Data['ROI'] = [region_of_interest(log) for log in Data['LogFile']]
Data['Duration'] = [duration(log) for log in Data['LogFile']]
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]

In [ ]:
# The iee research storage folder contains some folders with scans done by Kassandra on a SkyScan1273.
# Exclude those, since they are not part of this study, we just looked at them to help her.
for c, row in Data.iterrows():
    if '1273' in row.Scanner:
        # print('Dropping %s from our dataframe' % row.LogFile[len(Root)+1:])
        Data.drop([c], inplace=True)
# Reset dataframe index
Data = Data.reset_index(drop=True)

In [ ]:
# The iee research storage folder contains folders with scans of only teeth, done as a small pilot study.
# Exclude those, since they are not part of this study.
for c, row in Data.iterrows():
    if 'Teeth' in row.Folder:
        # print('Dropping %s from our dataframe' % row.LogFile[len(Root)+1:])
        Data.drop([c], inplace=True)
# Reset dataframe index
Data = Data.reset_index(drop=True)

In [ ]:
# Sort dataframe on fishes and scans
Data.sort_values(by=['Fish', 'Scan'], inplace=True)
# Reset dataframe index
Data = Data.reset_index(drop=True)

In [ ]:
# How many fishes did we scan?
# We scanned six 'BucketOfFish' so subtract those :)
print('We have %s unique names in our corpus of scans' % (len(Data.Fish.unique()) - 6))
print('We performed %s scans in total' % len(Data.Scan))

In [ ]:
Data['Total Duration'] = [st * stk for st, stk in zip(Data['Duration'], Data['Stacks'])]

In [ ]:
# Get an overview of the total scanning time
# Nice output based on https://stackoverflow.com/a/8907407/323100
total_seconds = int(Data['Total Duration'].sum())
hours, remainder = divmod(total_seconds, 60 * 60)
minutes, seconds = divmod(remainder, 60)
print('In total, we scanned for %s hours and %s minutes' % (hours, minutes))
for machine in Data['Scanner'].unique():
    total_seconds = int(Data[Data['Scanner'] == machine]['Total Duration'].sum())
    hours, remainder = divmod(total_seconds, 60 * 60)
    minutes, seconds = divmod(remainder, 60)
    print('\t - Of these, we scanned %s hours and %s minutes on the %s,'
          ' for %s scans' % (hours,
                             minutes,
                             machine,
                             len(Data[Data['Scanner'] == machine])))

In [ ]:
# We scanned six 'buckets of fish', so subtract those :)
print('We scanned %0.f fishes' % (len(Data.Fish.unique()) - 6))

In [ ]:
print('We did a total of %s scans' % len(Data))

In [ ]:
print('We perfomed %s scans with "head" in their folder name' % len(Data[Data['Scan'].str.contains('head')]))

In [ ]:
# for c, st in enumerate(['darkgrid',
#                         'whitegrid',
#                         'dark',
#                         'white',
#                         'ticks']):
#     seaborn.set_style(style=st)    
#     seaborn.boxenplot(Data.Voxelsize, color='#E6002E')
#     plt.ylim([0,50])
#     plt.title(st)   
#     plt.show()

In [ ]:
MikkisFile = sorted(glob.glob(os.path.join(Root, '*CTscanFishList.xlsx')))[0]
# Read excel file and use the first column as index
print('Reading in %s' % MikkisFile)
DataMikki = pandas.read_excel(MikkisFile)

In [ ]:
for i in DataMikki:
    print(i)

In [ ]:
DataMikki['Length(cm)'].unique()

In [ ]:
# Massage some values, so we can plot *all* values nicely
DataMikki['Length(cm)'].replace('?', numpy.nan, inplace=True)
DataMikki['Length(cm)'].replace('measure SL', numpy.nan, inplace=True)
DataMikki['Length(cm)'].replace('< 6', 5.5, inplace=True)
DataMikki['Length(cm)'].replace('< 7', 6.5, inplace=True)
DataMikki['Length(cm)'].replace('LE < 7', 6.5, inplace=True)

In [ ]:
sorted(DataMikki['Length(cm)'].unique())

Plot the length of all the fish, so we can 'visualize' them for the audience.
In one of the last seminars, a discussion sprung up about violin plots, so we deliberately show them here again :)

In [ ]:
# Plot fish lengths
seaborn.scatterplot(DataMikki['Length(cm)'])
plt.show()

In [ ]:
# Plot fish lengths
numpy.random.seed(1796)
seaborn.stripplot(DataMikki['Length(cm)'],
                  color='#E6002E',
                  size=plt.gca().yaxis.label.get_fontsize() * 0.618,
                  linewidth=1,
                  edgecolor='#cccccc',
                  jitter=0.8/2)
plt.ylim([0,21])
plt.ylabel('Length [cm]')
seaborn.despine()
plt.tight_layout()
plt.savefig(os.path.join(OutPutDir, 'lengths.plot.png'))

In [ ]:
# Plot fish lengths
numpy.random.seed(1796)
seaborn.stripplot(DataMikki['Length(cm)'],
                  color='#E6002E',
                  size=plt.gca().yaxis.label.get_fontsize() * 0.618,
                  linewidth=1,
                  edgecolor='#cccccc',
                  jitter=0.8/2)
seaborn.violinplot(DataMikki['Length(cm)'],
                   color='#E6002E',
                   edgecolor='#cccccc',
                   width=0.8,
                   saturation=1)
plt.ylim([0,21])
plt.ylabel('Length [cm]')
seaborn.despine()
plt.tight_layout()
plt.savefig(os.path.join(OutPutDir, 'lengths.violinplot.png'))

In [ ]:
# Plot fish lengths
numpy.random.seed(1796)
seaborn.stripplot(DataMikki['Length(cm)'],
                  color='#E6002E',
                  size=plt.gca().yaxis.label.get_fontsize() * 0.618,
                  linewidth=1,
                  edgecolor='#cccccc',
                  jitter=0.8/2)
seaborn.boxplot(DataMikki['Length(cm)'],
                color='#E6002E',
                linecolor='#cccccc',
                saturation=1)
plt.ylim([0, 21])
plt.ylabel('Length [cm]')
seaborn.despine()
plt.tight_layout()
plt.savefig(os.path.join(OutPutDir, 'lengths.boxplot.png'))

In [ ]:
# Plot fish lengths
numpy.random.seed(1796)
seaborn.stripplot(DataMikki['Length(cm)'],
                  color='#E6002E',
                  size=plt.gca().yaxis.label.get_fontsize() * 0.618,
                  linewidth=1,
                  edgecolor='#cccccc',
                  jitter=0.8/2)
seaborn.boxenplot(DataMikki['Length(cm)'],
                  color='#E6002E',
                  edgecolor='#cccccc',
                  line_kws=dict(color="#cccccc"),
                  saturation=1, showfliers=False)
plt.ylim([0, 21])
plt.ylabel('Length [cm]')
seaborn.despine()
plt.tight_layout()
plt.savefig(os.path.join(OutPutDir, 'lengths.boxenplot.png'))
plt.show()

In [ ]:
# Plot fish lengths
seaborn.boxenplot(DataMikki['Length(cm)'],
                  color='#E6002E',
                  edgecolor='#cccccc',
                  line_kws=dict(color="#cccccc"),
                  saturation=1)
plt.ylim([0, 21])
plt.ylabel('Length [cm]')
seaborn.despine()
plt.tight_layout()
plt.savefig(os.path.join(OutPutDir, 'lengths.boxenplot.only.png'))
plt.show()

In [ ]:
# Plot scan voxel size
seaborn.boxenplot(Data['Voxelsize'],
                  color='#E6002E',
                  edgecolor='#cccccc',
                  line_kws=dict(color="#cccccc"),
                  saturation=1)
plt.ylabel('Voxelsize [μm]')
plt.ylim([0, 51])
seaborn.despine()
plt.tight_layout()
plt.savefig(os.path.join(OutPutDir, 'voxelsizes.png'))
plt.show()

In [ ]:
# Let's only talk about one fish
fish = '104016'

In [ ]:
# Get rid of all other fish, so we can easily see what we talk about
for c, row in Data.iterrows():
    if fish not in row.Fish:
        Data.drop([c], inplace=True)
Data = Data.reset_index(drop=True)

In [ ]:
# Data[Data.isin([fish])]
Data

In [ ]:
    # In which jar should it be/go?
    foundfishes = 0
    for d, row in DataMikki.iterrows():
        if (str(fish).lower() in str(row.Fishec).lower()) \
        or (str(fish).lower() in str(row.FieldID).lower()) \
        or (str(fish).lower() in str(row.OtherID).lower()) \
        or (str(fish).lower() in str(row.ReplacementID).lower()):
            foundfishes = (row.Fishec, row.FieldID, row.OtherID, row.ReplacementID)
            # remove nan from the list of hits
            foundfishes = [str(x).lower() for x in foundfishes if not pandas.isnull(x)]
            print('*%s*: The fish ' % fish, end='')
            if len(foundfishes) > 1:
                for found in foundfishes:
                    print(found.upper(), end='/')
            else:
                print(foundfishes[0].upper(), end='')
            print(' should now go in jar "length=%s cm" (%s))' % (row['Length(cm)'],
                                                                  row['TemporaryJar']))
    if not foundfishes:
        print('*%s*: Nothing found in %s' % (fish, MikkisFile))

In [ ]:
    # Do we have something from this fish on disk?
    ondisk = glob.glob(os.path.join(Root, '*%s*' % fish))
    if len(ondisk):
        for found in ondisk:
            print('*%s*: Found on disk in %s' % (fish, found))
            foundondisk = 1
    else:
        print('*%s*: Nothing found in %s' % (fish, Root))
        foundondisk = 0

In [ ]:
    # Did we scan it already?
    found = 0
    for c, row in Data.iterrows():
        if fish in row.Fish:
            print('*%s*: Sample %s/%s was scanned on %s' % (fish, row['Fish'], row['Scan'], row['Scan date']))
            found = 1
    if not found:
        if foundondisk:
            print('*%s*: We have a folder (%s) for this sample, but nothing in the dataframe, so it probably is all good' % (fish, ondisk[0]))
            print('Check the folder to be shure')
        else:
            print('*%s*: Nothing about this sample is found in our dataframe' % fish)